# Varias utilidades con hojas Excel
### List of sheets in an Excel file
### Excel sheet to csv changing the encoding
### Create table (sql) from a dataframe

In [3]:
import pandas as pd
import numpy as np # to convert types
# You can use dtype or convertwers arguments when read the Excel
# for example: 
#      converters={'id': str}
#      dtype={'Col1': np.float32, 'Col2': np.int32}

## List of sheets in an Excel file

In [2]:
fi = r'D:\IGME20\20201215_aquifer\_data\JLGA_20210217\CRCC\CRCC_EIA2017.xlsx'

In [3]:
# Remenber you can use dtype or convertwers arguments when read the Excel
df = pd.read_excel(fi, 
                   sheet_name=None, 
                   engine='openpyxl',
                  )
for key in df.keys():
    print(key)

VolsFinales
DatosCHS
Peticiones20170112
EIA
EIAconDatosCHS
Hoja2


## Excel to csv changing the encoding

In [46]:
# Reads Excel sheet (you can change sheet column types using dtype or converters)
df = pd.read_excel(fi, 
                   sheet_name=sheet_name, 
                   engine='openpyxl'
                  )
df.to_csv(fo, index=False, encoding='utf-8')

## Write a basic create table command from df 

In [12]:
fi = r'D:\IGME20\20201215_aquifer\_data\JLGA_20210217\CRCC\CRCC_EIA2017.xlsx'
sheet_name = 'EIAconDatosCHS'
output = r'D:\IGME20\20201215_aquifer\_data\JLGA_20210217\CRCC\crcc_eia2017.sql'

In [17]:
# Remenber you can use dtype or convertwers arguments when read the Excel
df = pd.read_excel(fi, sheet_name=sheet_name, engine='openpyxl',
                    converters={'cod_igme': str, 'vol_solicitado_m3': float, 
                                'UTMX': float, 'UTMY': float})

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gid                252 non-null    int64  
 1   cod_igme           0 non-null      object 
 2   nomacu             252 non-null    object 
 3   expediente         252 non-null    object 
 4   peticionario       252 non-null    object 
 5   prov               252 non-null    object 
 6   tm                 252 non-null    object 
 7   vol_solicitado_m3  252 non-null    float64
 8   vol_corregido_chs  252 non-null    float64
 9   captacion          252 non-null    object 
 10  tipo_en_chs        249 non-null    object 
 11  prof_m             251 non-null    float64
 12  qmax_ls            223 non-null    float64
 13  bomba_kw           223 non-null    float64
 14  has_atendidas      169 non-null    float64
 15  UTMX               252 non-null    float64
 16  UTMY               252 non

In [19]:
df.head(3)

,gid,cod_igme,nomacu,expediente,peticionario,prov,tm,vol_solicitado_m3,vol_corregido_chs,captacion,tipo_en_chs,prof_m,qmax_ls,bomba_kw,has_atendidas,UTMX,UTMY
0,1,NaN,Cuaternario,APV-83/2016,"ROCA INGLES, FRANCISCO",MURCIA,Fuente Álamo,3000.0,3000.0,A construir,NaN,NaN,NaN,NaN,NaN,659106.0,4174112.0
1,2,NaN,Plioceno,APV-90/2016,PAREDES PEDREÑO ANDRES,MURCIA,Cartagena,58899.0,58899.0,Existente bombeó en el pasado,Sondeo con desalobradora,90.0,7.0,13.4,19.6329,682212.0,4171578.0
2,3,NaN,Plioceno,APV-91/2016,"INGLES DOMENECH, JOSÉ",MURCIA,Cartagena,50000.0,50000.0,Existente bombeó en el pasado,Sondeo/s*,90.0,5.0,15.0,NaN,684360.0,4171404.0


In [42]:
table_name = 'crcc_eia2017'
ident = ' ' * 4

In [43]:
# Relationship between current types anf postgres types
# For a specific column conversion (for example object to integer), do that when the Excel is read)
convert_types={'object':'varchar', 'int64': 'integer', 'float64': 'real'}
db_types = [convert_types[str(type1)] for type1 in df.dtypes if str(type1) in convert_types]
if len(df.dtypes) == len(db_types):
    print('OK, all df.dtypes are in convert_types')
else:
    for type1 in df.dtypes:
        if type1 not in convert_types:
            print(f'{type1} has not a db type, check convert_types')

OK, all df.dtypes are in convert_types


In [44]:
# write sql output file to creata the table
columns = [f'{ident}{column1} {type1}' for column1, type1 in zip(df.columns, db_types)]
columns = ',\n'.join(columns)
with open(output, 'w') as fo:
    fo.write(f'create table {table_name} (\n')
    fo.write(f'{columns}')
    fo.write('\n);\n')
print(f'Go to {output}\nand review it')

Go to D:\IGME20\20201215_aquifer\_data\JLGA_20210217\CRCC\crcc_eia2017.sql
and review it
